In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow import keras

import os

fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

# normalization: x = (x - u) / std (其中u为训练集均值，std为训练集方差)
scaler = StandardScaler()
# x_train: [None, 28m 28] -> [None, 784] -> [None, 28, 28]
# fit_transform中fit的作用: 记录下数据集的均值和方差 (由于在做normalization时使用的均值和方差是训练集上的均值和方差，所以需要记录下来)
x_train_scaled = scaler.fit_transform(x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_valid_scaled = scaler.transform(x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)
x_test_scaled = scaler.transform(x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28, 28)




print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

def show_single_image(img_arr):
    plt.imshow(img_arr, cmap='binary')
    plt.show()

def show_imgs(n_rows, n_cols, x_data, y_data, class_names):
    assert len(x_data) == len(y_data)
    assert n_rows*n_cols < len(x_data)
    plt.figure(figsize=(n_cols*1.4, n_rows*1.6))
    for row in range(n_rows):
        for col in range(n_cols):
            index = n_cols*row+col
            plt.subplot(n_rows, n_cols, index+1)
            plt.imshow(x_data[index], cmap='binary', interpolation='nearest')
            plt.axis('off')
            plt.title(class_names[y_data[index]])
    plt.show()


# show_single_image(x_train[0])
# class_names = ['T-shirt', 'Touser', 'Pullover', 'Dress', 'Coat', 'Sandal',
#                'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# show_imgs(3, 5, x_train, y_train, class_names)

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
model.add(keras.layers.Dense(300, activation='relu'))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax')) #将向量变成概率分布

# reason for sparse: y->one_hot->向量
# 若y是一个向量类似[0.1, 0.2, 0.7]则使用categorical_crossentropy
# 若y是一个数通过one_hot转换成向量则使用sparse_categorical_crossentropy
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()   #打出模型架构

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


## 保存与加载

### 保存整个模型

In [5]:
# callback 回调函数
# TensorBoard, EarlyStopping, ModelCheckpoint
logdir = './callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, 'fashion_minst_model.h5')

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file, save_best_only = True),
]
history = model.fit(x_train_scaled, y_train, epochs=10, validation_data=(x_valid_scaled, y_valid), callbacks = callbacks)


loaded_model = keras.models.load_model(output_model_file)

loaded_model.evaluate(x_test_scaled, y_test)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
   32/55000 [..............................] - ETA: 2:05 - loss: 1.3101 - accuracy: 0.5312

ProfilerNotRunningError: Cannot stop profiling. No profiler is running.

### 只保存weights

### 方法一：使用callbacks --- checkpoint

In [ ]:
# 只保存weights在加载模型时需要手动构建模型，然后加载weights
logdir = './callbacks'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir, 'fashion_minst_weights.h5')

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file, save_best_only = True, save_weights_only = True),
    keras.callbacks.EarlyStopping(patience=5, min_delta=1e-3),
]
history = model.fit(x_train_scaled, y_train, epochs=10, validation_data=(x_valid_scaled, y_valid), callbacks = callbacks)


# show_single_image(x_train[0])
# class_names = ['T-shirt', 'Touser', 'Pullover', 'Dress', 'Coat', 'Sandal',
#                'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
# show_imgs(3, 5, x_train, y_train, class_names)

loaded_model = keras.models.Sequential()
loaded_model.add(keras.layers.Flatten(input_shape=[28, 28]))
loaded_model.add(keras.layers.Dense(300, activation='relu'))
loaded_model.add(keras.layers.Dense(100, activation='relu'))
loaded_model.add(keras.layers.Dense(10, activation='softmax')) #将向量变成概率分布

loaded_model.load_weights(output_model_file)

# reason for sparse: y->one_hot->向量
# 若y是一个向量类似[0.1, 0.2, 0.7]则使用categorical_crossentropy
# 若y是一个数通过one_hot转换成向量则使用sparse_categorical_crossentropy
loaded_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()   #打出模型架构

loaded_model.evaluate(x_test_scaled, y_test)

### 方法二: 使用model的save_weights方法

In [ ]:
model.save_weights(os.path.join(logdir, 'fashion_minst_weights.h5'))